# SWAMpy Main Call





- Utility functions used to populate dictionaries and arrays required to be passed to core SAMBUCA inversion algorithm
- Combination of hardcoded and local path calls are currently required, with the 'end goal' of defining XMLs to pass straight to these dictoniaries, dervied by any API/method (manual, SNAP GUI, AGDC API etc)
- Processing and Output options are hardcoded in corresponding modules as noted.

Developments to Date that extend the original Brando et. al (2009) SAMBUCA implementation:
- Inclusion of 3 substrates in the inversion
- The option to relax the sum-to-one constraint on the substrate composition (see Petit et. al. (2017))
- Option of selecting bands with which to use in the inversion

NOTE: Sambuca modules as well as Sen2Coral utility scripts have been amended from the SAMBUCA implementation due to a mismatch between how the array dimension is represented. The image data is read to an array (bands, rows, columns), however the original sambuca code treats this as (bands, x, y) or (bands, width, height). Original testing on a square image did not highlight this bug. The original sambuca repository needs to be fixed still.

# Set up the environment



In [1]:
import numpy as np
import sys

#import Marcos script modules and set up path to these

sys.path.append('C:\\Users\\PCUSER\\sambuca_project\\SWAMpy\\sen2coral')

import sambuca_input_rrs
import sambuca_input_parameters
import sambuca_preparation
import sambuca_calculations
import sambuca_outputs
import define_outputs


# set some controls on numpy formatting
# 5 decimal places, suppress scientific notation
np.set_printoptions(precision=5, suppress=True)

Now import the sambuca and sambuca-core packages:

In [2]:
import sambuca as sb
import sambuca_core as sbc

In [3]:
# Main statement needed if exporting to .py script. 
#if __name__=='__main__':

# Define a base project input path

In this path the code expects the following folders
- image
- siop
- nedr
- substrates
- sensor_filters

This path can used to point to XML files to ingest when ready for that. At the moment, files specified in the input folders are hardcoded in sambuca_input_rrs and sambuca_input_parameters modules.

In [4]:
base_path = 'C:\\Users\\PCUSER\\sambuca_project\\SWAMpy\\input_data\\'

# Load the Input Image Data
*sambuca_input_rrs.py*

- The code requires below-surface remote sensing reflectance (rrs)
- The input data is loaded into an observed_rrs array of shape (bands, rows, columns) using rasterio. 
- Other image metadata is loaded into a tagged image_info dictionary based on rasterio src class. (e.g crs, affine, width, height)
- Full sensor filter dictonary is loaded, and specific sensor filter extracted based on sensor id key
- If Rrs is set to True, the user is supplying above-surface remote sensing reflectance. This is coverted to below-surface rrs if True is set. Default value if not set is False
- There is a hardcoded option to select a sub-set of bands to run the inversion on. The user must ensure that the sensor filter is then of the same size. For example: S25 for 5 bands of Sentinel-2. The user must also make a sensible choice based on the minimum common wavelengths of the other input spectra (substrates, a_ph_star, awater) (see the Prepare the Data call)
- Noise Equivilant (nedr) is also loaded here, with the path specified. Similar to the sensor filter, the user must also make sure the number of bands in the nedr selected matches that of the selected bands in the input data.

File names and sensor id key are currently hardcoded.

In [5]:
[observed_rrs, image_info]=sambuca_input_rrs.sam_obs(base_path, Rrs = False)

Observed rrs file:  C:\Users\PCUSER\sambuca_project\SWAMpy\input_data\image\LS84bandsGBRSub_rrs.img
Width, height:  66 88
crs:  CRS({'proj': 'utm', 'south': True, 'zone': 55, 'no_defs': True, 'units': 'm', 'ellps': 'GRS80'})
affine:  | 25.00, 0.00, 644375.00|
| 0.00,-25.00, 7877950.00|
| 0.00, 0.00, 1.00|
num bands:  4
band indicies:  (1, 2, 3, 4)


C:\Users\PCUSER\sambuca_project\WinPython-64bit-3.5.3.1Qt5\python-3.5.3.amd64\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  if __name__ == '__main__':
C:\Users\PCUSER\sambuca_project\WinPython-64bit-3.5.3.1Qt5\python-3.5.3.amd64\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  if __name__ == '__main__':


(4, 88, 66)
(4, 88, 66)


# Load the Parameters
*sambuca_input_parameters.py*

Builds Dictionaries to pass to the data prep and inversion 
- SIOPs: spectral, values, substrates, free parameter bounds
- Enviromental data: sun angle. off nadir, q factor

Currently hardcoded and read in from input data folder, aim to build from XML

Bounds for the three substrates here should reflect if the user is using a sum to one constraint, or the relaxed constraint (RASC) (specified in the Run the Inversion step).

- If Sum-to-One, lower bounds = 0  and upper bounds = 1 for each substrate
- For RASC, lower bounds = 0 and upper bounds = upper limit of RASC.



In [6]:
[siop, envmeta]=sambuca_input_parameters.sam_par(base_path)

# Prepare the Data
*sambuca_preparation.py*

- Prepares the spectral inputs (currently truncates to a common wavelength range).
- Builds a fixed parameter set out of truncated inputs, siop and envdata dictionaries.
- Prepares a result recorder for storing array based results.
- Defines the objective function (Hardcoded call currently made to specific error function here)

In [7]:
[wavelengths, siop, image_info, fixed_parameters, result_recorder, objective]=sambuca_preparation.sam_prep(siop, envmeta, 
                                                                                                           image_info)

Common wavelength range: 350.0 - 900.0
awater: min: 350  max: 900
a_ph_star: min: 350  max: 900
hi_3:sand: min: 350.0  max: 900.0
hi_3:Acropora: min: 350.0  max: 900.0
hi_3:Turf Algae: min: 350.0  max: 900.0


# Run the Inversion
*sambuca_calculations.py*

Sets up and runs the inversion:
- Type of SciPy optimisation used
    - For RASC application, perhaps only SLSQP is appropriate
- Definition of an image subset to be processed if required (primarily for testing; as full result recorder array extent of original image is still used, and crs/affine may not translate when written to geotiff in outputs)
- Parameter starting points (currently mid point of range)
- Option to select and set relaxed constraints on substrate abundances (RASC), or to maintain sum-to-one
    - Suitable substrate parameter bounds should also be set (see Load the Parameters)
- Option for SAMBUCA option of 'go shallow', when set to true, retrieves as shallow depth as possible for optically deep waters whilst maintaining an SDI value of < 1.

Returns the results, spatial subset range processed, and number of pixels processed

In [8]:
[result_recorder, coordinates, num_pixels]=sambuca_calculations.sam_com(observed_rrs, objective, siop,
                                                                        result_recorder, image_info, shallow = False)

[0, 0]
[0, 1]
[0, 2]
[0, 3]
[0, 4]
[0, 5]
[0, 6]
[0, 7]
[0, 8]
[0, 9]
[0, 10]
[0, 11]
[0, 12]
[0, 13]
[0, 14]
[0, 15]
[0, 16]
[0, 17]
[0, 18]
[0, 19]
[0, 20]
[0, 21]
[0, 22]
[0, 23]
[0, 24]
[0, 25]
[0, 26]
[0, 27]
[0, 28]
[0, 29]
[0, 30]
[0, 31]
[0, 32]
[0, 33]
[0, 34]
[0, 35]
[0, 36]
[0, 37]
[0, 38]
[0, 39]
[0, 40]
[0, 41]
[0, 42]
[0, 43]
[0, 44]
[0, 45]
[0, 46]
[0, 47]
[0, 48]
[0, 49]
[0, 50]
[0, 51]
[0, 52]
[0, 53]
[0, 54]
[0, 55]
[0, 56]
[0, 57]
[0, 58]
[0, 59]
[0, 60]
[0, 61]
[0, 62]
[0, 63]
[0, 64]
[0, 65]
[1, 0]
[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[1, 7]
[1, 8]
[1, 9]
[1, 10]
[1, 11]
[1, 12]
[1, 13]
[1, 14]
[1, 15]
[1, 16]
[1, 17]
[1, 18]
[1, 19]
[1, 20]
[1, 21]
[1, 22]
[1, 23]
[1, 24]
[1, 25]
[1, 26]
[1, 27]
[1, 28]
[1, 29]
[1, 30]
[1, 31]
[1, 32]
[1, 33]
[1, 34]
[1, 35]
[1, 36]
[1, 37]
[1, 38]
[1, 39]
[1, 40]
[1, 41]
[1, 42]
[1, 43]
[1, 44]
[1, 45]
[1, 46]
[1, 47]
[1, 48]
[1, 49]
[1, 50]
[1, 51]
[1, 52]
[1, 53]
[1, 54]
[1, 55]
[1, 56]
[1, 57]
[1, 58]
[1, 59]
[1, 60]
[1, 

[15, 12]
[15, 13]
[15, 14]
[15, 15]
[15, 16]
[15, 17]
[15, 18]
[15, 19]
[15, 20]
[15, 21]
[15, 22]
[15, 23]
[15, 24]
[15, 25]
[15, 26]
[15, 27]
[15, 28]
[15, 29]
[15, 30]
[15, 31]
[15, 32]
[15, 33]
[15, 34]
[15, 35]
[15, 36]
[15, 37]
[15, 38]
[15, 39]
[15, 40]
[15, 41]
[15, 42]
[15, 43]
[15, 44]
[15, 45]
[15, 46]
[15, 47]
[15, 48]
[15, 49]
[15, 50]
[15, 51]
[15, 52]
[15, 53]
[15, 54]
[15, 55]
[15, 56]
[15, 57]
[15, 58]
[15, 59]
[15, 60]
[15, 61]
[15, 62]
[15, 63]
[15, 64]
[15, 65]
[16, 0]
[16, 1]
[16, 2]
[16, 3]
[16, 4]
[16, 5]
[16, 6]
[16, 7]
[16, 8]
[16, 9]
[16, 10]
[16, 11]
[16, 12]
[16, 13]
[16, 14]
[16, 15]
[16, 16]
[16, 17]
[16, 18]
[16, 19]
[16, 20]
[16, 21]
[16, 22]
[16, 23]
[16, 24]
[16, 25]
[16, 26]
[16, 27]
[16, 28]
[16, 29]
[16, 30]
[16, 31]
[16, 32]
[16, 33]
[16, 34]
[16, 35]
[16, 36]
[16, 37]
[16, 38]
[16, 39]
[16, 40]
[16, 41]
[16, 42]
[16, 43]
[16, 44]
[16, 45]
[16, 46]
[16, 47]
[16, 48]
[16, 49]
[16, 50]
[16, 51]
[16, 52]
[16, 53]
[16, 54]
[16, 55]
[16, 56]
[16, 57]
[1

[29, 15]
[29, 16]
[29, 17]
[29, 18]
[29, 19]
[29, 20]
[29, 21]
[29, 22]
[29, 23]
[29, 24]
[29, 25]
[29, 26]
[29, 27]
[29, 28]
[29, 29]
[29, 30]
[29, 31]
[29, 32]
[29, 33]
[29, 34]
[29, 35]
[29, 36]
[29, 37]
[29, 38]
[29, 39]
[29, 40]
[29, 41]
[29, 42]
[29, 43]
[29, 44]
[29, 45]
[29, 46]
[29, 47]
[29, 48]
[29, 49]
[29, 50]
[29, 51]
[29, 52]
[29, 53]
[29, 54]
[29, 55]
[29, 56]
[29, 57]
[29, 58]
[29, 59]
[29, 60]
[29, 61]
[29, 62]
[29, 63]
[29, 64]
[29, 65]
[30, 0]
[30, 1]
[30, 2]
[30, 3]
[30, 4]
[30, 5]
[30, 6]
[30, 7]
[30, 8]
[30, 9]
[30, 10]
[30, 11]
[30, 12]
[30, 13]
[30, 14]
[30, 15]
[30, 16]
[30, 17]
[30, 18]
[30, 19]
[30, 20]
[30, 21]
[30, 22]
[30, 23]
[30, 24]
[30, 25]
[30, 26]
[30, 27]
[30, 28]
[30, 29]
[30, 30]
[30, 31]
[30, 32]
[30, 33]
[30, 34]
[30, 35]
[30, 36]
[30, 37]
[30, 38]
[30, 39]
[30, 40]
[30, 41]
[30, 42]
[30, 43]
[30, 44]
[30, 45]
[30, 46]
[30, 47]
[30, 48]
[30, 49]
[30, 50]
[30, 51]
[30, 52]
[30, 53]
[30, 54]
[30, 55]
[30, 56]
[30, 57]
[30, 58]
[30, 59]
[30, 60]
[3

[43, 18]
[43, 19]
[43, 20]
[43, 21]
[43, 22]
[43, 23]
[43, 24]
[43, 25]
[43, 26]
[43, 27]
[43, 28]
[43, 29]
[43, 30]
[43, 31]
[43, 32]
[43, 33]
[43, 34]
[43, 35]
[43, 36]
[43, 37]
[43, 38]
[43, 39]
[43, 40]
[43, 41]
[43, 42]
[43, 43]
[43, 44]
[43, 45]
[43, 46]
[43, 47]
[43, 48]
[43, 49]
[43, 50]
[43, 51]
[43, 52]
[43, 53]
[43, 54]
[43, 55]
[43, 56]
[43, 57]
[43, 58]
[43, 59]
[43, 60]
[43, 61]
[43, 62]
[43, 63]
[43, 64]
[43, 65]
[44, 0]
[44, 1]
[44, 2]
[44, 3]
[44, 4]
[44, 5]
[44, 6]
[44, 7]
[44, 8]
[44, 9]
[44, 10]
[44, 11]
[44, 12]
[44, 13]
[44, 14]
[44, 15]
[44, 16]
[44, 17]
[44, 18]
[44, 19]
[44, 20]
[44, 21]
[44, 22]
[44, 23]
[44, 24]
[44, 25]
[44, 26]
[44, 27]
[44, 28]
[44, 29]
[44, 30]
[44, 31]
[44, 32]
[44, 33]
[44, 34]
[44, 35]
[44, 36]
[44, 37]
[44, 38]
[44, 39]
[44, 40]
[44, 41]
[44, 42]
[44, 43]
[44, 44]
[44, 45]
[44, 46]
[44, 47]
[44, 48]
[44, 49]
[44, 50]
[44, 51]
[44, 52]
[44, 53]
[44, 54]
[44, 55]
[44, 56]
[44, 57]
[44, 58]
[44, 59]
[44, 60]
[44, 61]
[44, 62]
[44, 63]
[4

[57, 21]
[57, 22]
[57, 23]
[57, 24]
[57, 25]
[57, 26]
[57, 27]
[57, 28]
[57, 29]
[57, 30]
[57, 31]
[57, 32]
[57, 33]
[57, 34]
[57, 35]
[57, 36]
[57, 37]
[57, 38]
[57, 39]
[57, 40]
[57, 41]
[57, 42]
[57, 43]
[57, 44]
[57, 45]
[57, 46]
[57, 47]
[57, 48]
[57, 49]
[57, 50]
[57, 51]
[57, 52]
[57, 53]
[57, 54]
[57, 55]
[57, 56]
[57, 57]
[57, 58]
[57, 59]
[57, 60]
[57, 61]
[57, 62]
[57, 63]
[57, 64]
[57, 65]
[58, 0]
[58, 1]
[58, 2]
[58, 3]
[58, 4]
[58, 5]
[58, 6]
[58, 7]
[58, 8]
[58, 9]
[58, 10]
[58, 11]
[58, 12]
[58, 13]
[58, 14]
[58, 15]
[58, 16]
[58, 17]
[58, 18]
[58, 19]
[58, 20]
[58, 21]
[58, 22]
[58, 23]
[58, 24]
[58, 25]
[58, 26]
[58, 27]
[58, 28]
[58, 29]
[58, 30]
[58, 31]
[58, 32]
[58, 33]
[58, 34]
[58, 35]
[58, 36]
[58, 37]
[58, 38]
[58, 39]
[58, 40]
[58, 41]
[58, 42]
[58, 43]
[58, 44]
[58, 45]
[58, 46]
[58, 47]
[58, 48]
[58, 49]
[58, 50]
[58, 51]
[58, 52]
[58, 53]
[58, 54]
[58, 55]
[58, 56]
[58, 57]
[58, 58]
[58, 59]
[58, 60]
[58, 61]
[58, 62]
[58, 63]
[58, 64]
[58, 65]
[59, 0]
[59

[71, 23]
[71, 24]
[71, 25]
[71, 26]
[71, 27]
[71, 28]
[71, 29]
[71, 30]
[71, 31]
[71, 32]
[71, 33]
[71, 34]
[71, 35]
[71, 36]
[71, 37]
[71, 38]
[71, 39]
[71, 40]
[71, 41]
[71, 42]
[71, 43]
[71, 44]
[71, 45]
[71, 46]
[71, 47]
[71, 48]
[71, 49]
[71, 50]
[71, 51]
[71, 52]
[71, 53]
[71, 54]
[71, 55]
[71, 56]
[71, 57]
[71, 58]
[71, 59]
[71, 60]
[71, 61]
[71, 62]
[71, 63]
[71, 64]
[71, 65]
[72, 0]
[72, 1]
[72, 2]
[72, 3]
[72, 4]
[72, 5]
[72, 6]
[72, 7]
[72, 8]
[72, 9]
[72, 10]
[72, 11]
[72, 12]
[72, 13]
[72, 14]
[72, 15]
[72, 16]
[72, 17]
[72, 18]
[72, 19]
[72, 20]
[72, 21]
[72, 22]
[72, 23]
[72, 24]
[72, 25]
[72, 26]
[72, 27]
[72, 28]
[72, 29]
[72, 30]
[72, 31]
[72, 32]
[72, 33]
[72, 34]
[72, 35]
[72, 36]
[72, 37]
[72, 38]
[72, 39]
[72, 40]
[72, 41]
[72, 42]
[72, 43]
[72, 44]
[72, 45]
[72, 46]
[72, 47]
[72, 48]
[72, 49]
[72, 50]
[72, 51]
[72, 52]
[72, 53]
[72, 54]
[72, 55]
[72, 56]
[72, 57]
[72, 58]
[72, 59]
[72, 60]
[72, 61]
[72, 62]
[72, 63]
[72, 64]
[72, 65]
[73, 0]
[73, 1]
[73, 2]
[73, 

[85, 26]
[85, 27]
[85, 28]
[85, 29]
[85, 30]
[85, 31]
[85, 32]
[85, 33]
[85, 34]
[85, 35]
[85, 36]
[85, 37]
[85, 38]
[85, 39]
[85, 40]
[85, 41]
[85, 42]
[85, 43]
[85, 44]
[85, 45]
[85, 46]
[85, 47]
[85, 48]
[85, 49]
[85, 50]
[85, 51]
[85, 52]
[85, 53]
[85, 54]
[85, 55]
[85, 56]
[85, 57]
[85, 58]
[85, 59]
[85, 60]
[85, 61]
[85, 62]
[85, 63]
[85, 64]
[85, 65]
[86, 0]
[86, 1]
[86, 2]
[86, 3]
[86, 4]
[86, 5]
[86, 6]
[86, 7]
[86, 8]
[86, 9]
[86, 10]
[86, 11]
[86, 12]
[86, 13]
[86, 14]
[86, 15]
[86, 16]
[86, 17]
[86, 18]
[86, 19]
[86, 20]
[86, 21]
[86, 22]
[86, 23]
[86, 24]
[86, 25]
[86, 26]
[86, 27]
[86, 28]
[86, 29]
[86, 30]
[86, 31]
[86, 32]
[86, 33]
[86, 34]
[86, 35]
[86, 36]
[86, 37]
[86, 38]
[86, 39]
[86, 40]
[86, 41]
[86, 42]
[86, 43]
[86, 44]
[86, 45]
[86, 46]
[86, 47]
[86, 48]
[86, 49]
[86, 50]
[86, 51]
[86, 52]
[86, 53]
[86, 54]
[86, 55]
[86, 56]
[86, 57]
[86, 58]
[86, 59]
[86, 60]
[86, 61]
[86, 62]
[86, 63]
[86, 64]
[86, 65]
[87, 0]
[87, 1]
[87, 2]
[87, 3]
[87, 4]
[87, 5]
[87, 6]


# Define and Write some Outputs
*define_outputs.py*

- Outputs types wanted are defined in define_outputs.py and call to a writeout function within sambuca_outputs.py
    - Examples can be extended. Some variables are calculated on the fly in here, for example, the total abundance of the three       RASC substrates proportions and the normalised RASC fractions.
- Written out as geotiffs with crs info inherited from the input rrs data
- Examples options of writing to png plots generated by MatPlotlib
- If a subsection of image to be processed was selected in sambuca_calculations, coordinates ensure only this section is written (validation needed to see if crs and affine info translates correctly for a subset write)

In [9]:
define_outputs.output_suite(result_recorder, image_info, coordinates)